<a href="https://colab.research.google.com/github/dempstermatthew/pbd/blob/master/Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
from google.colab import files
uploaded = files.upload()

Saving alcohol_list.csv to alcohol_list.csv


In [0]:
df = spark.read.csv("alcohol_list.csv", inferSchema=True, header=True)

In [6]:
type(df)

pyspark.sql.dataframe.DataFrame

In [7]:
df.describe()

DataFrame[summary: string, GHO (CODE): string, GHO (DISPLAY): string, GHO (URL): string, DATASOURCE (CODE): string, DATASOURCE (DISPLAY): string, DATASOURCE (URL): string, PUBLISHSTATE (CODE): string, PUBLISHSTATE (DISPLAY): string, PUBLISHSTATE (URL): string, YEAR (CODE): string, YEAR (DISPLAY): string, YEAR (URL): string, REGION (CODE): string, REGION (DISPLAY): string, REGION (URL): string, COUNTRY (CODE): string, COUNTRY (DISPLAY): string, COUNTRY (URL): string, ALCOHOLTYPE (CODE): string, ALCOHOLTYPE (DISPLAY): string, ALCOHOLTYPE (URL): string, Display Value: string, Numeric: string, Low: string, High: string, StdErr: string, StdDev: string, Comments: string]

In [8]:
df.show()

+-------------+--------------------+--------------------+-----------------+--------------------+----------------+-------------------+----------------------+------------------+-----------+--------------+----------+-------------+--------------------+------------+--------------+--------------------+-------------+------------------+---------------------+-----------------+-------------+-------+----+----+------+------+--------+
|   GHO (CODE)|       GHO (DISPLAY)|           GHO (URL)|DATASOURCE (CODE)|DATASOURCE (DISPLAY)|DATASOURCE (URL)|PUBLISHSTATE (CODE)|PUBLISHSTATE (DISPLAY)|PUBLISHSTATE (URL)|YEAR (CODE)|YEAR (DISPLAY)|YEAR (URL)|REGION (CODE)|    REGION (DISPLAY)|REGION (URL)|COUNTRY (CODE)|   COUNTRY (DISPLAY)|COUNTRY (URL)|ALCOHOLTYPE (CODE)|ALCOHOLTYPE (DISPLAY)|ALCOHOLTYPE (URL)|Display Value|Numeric| Low|High|StdErr|StdDev|Comments|
+-------------+--------------------+--------------------+-----------------+--------------------+----------------+-------------------+---------------

In [9]:
df.printSchema()

root
 |-- GHO (CODE): string (nullable = true)
 |-- GHO (DISPLAY): string (nullable = true)
 |-- GHO (URL): string (nullable = true)
 |-- DATASOURCE (CODE): string (nullable = true)
 |-- DATASOURCE (DISPLAY): string (nullable = true)
 |-- DATASOURCE (URL): string (nullable = true)
 |-- PUBLISHSTATE (CODE): string (nullable = true)
 |-- PUBLISHSTATE (DISPLAY): string (nullable = true)
 |-- PUBLISHSTATE (URL): string (nullable = true)
 |-- YEAR (CODE): integer (nullable = true)
 |-- YEAR (DISPLAY): integer (nullable = true)
 |-- YEAR (URL): string (nullable = true)
 |-- REGION (CODE): string (nullable = true)
 |-- REGION (DISPLAY): string (nullable = true)
 |-- REGION (URL): string (nullable = true)
 |-- COUNTRY (CODE): string (nullable = true)
 |-- COUNTRY (DISPLAY): string (nullable = true)
 |-- COUNTRY (URL): string (nullable = true)
 |-- ALCOHOLTYPE (CODE): string (nullable = true)
 |-- ALCOHOLTYPE (DISPLAY): string (nullable = true)
 |-- ALCOHOLTYPE (URL): string (nullable = true)
 

In [10]:
df.count()

13235

In [0]:
df_whatever = df.select('YEAR (DISPLAY)', 'REGION (DISPLAY)', 'COUNTRY (DISPLAY)', 'ALCOHOLTYPE (DISPLAY)', 'Display Value')

In [12]:
df_whatever.first()

Row(YEAR (DISPLAY)=2011, REGION (DISPLAY)='Western Pacific', COUNTRY (DISPLAY)='Australia', ALCOHOLTYPE (DISPLAY)='Wine', Display Value='3.79')

In [13]:
df_whatever.describe('Display Value').show()

+-------+------------------+
|summary|     Display Value|
+-------+------------------+
|  count|             13235|
|   mean| 2.022251300887662|
| stddev|2.7920093615709276|
|    min|                 0|
|    max|           No data|
+-------+------------------+



In [14]:

df_whatever.filter((df['COUNTRY (DISPLAY)']=='Ireland') & (df['YEAR (DISPLAY)']==2014)).show()

+--------------+----------------+-----------------+---------------------+-------------+
|YEAR (DISPLAY)|REGION (DISPLAY)|COUNTRY (DISPLAY)|ALCOHOLTYPE (DISPLAY)|Display Value|
+--------------+----------------+-----------------+---------------------+-------------+
|          2014|          Europe|          Ireland|                 Beer|         5.13|
|          2014|          Europe|          Ireland|            All types|        10.75|
|          2014|          Europe|          Ireland|              Spirits|         1.97|
|          2014|          Europe|          Ireland| Other alcoholic b...|         0.85|
|          2014|          Europe|          Ireland|                 Wine|         2.80|
+--------------+----------------+-----------------+---------------------+-------------+



In [15]:
df_whatever.where((df['COUNTRY (DISPLAY)']=='Ireland') & (df['YEAR (DISPLAY)']==2014)).show()

+--------------+----------------+-----------------+---------------------+-------------+
|YEAR (DISPLAY)|REGION (DISPLAY)|COUNTRY (DISPLAY)|ALCOHOLTYPE (DISPLAY)|Display Value|
+--------------+----------------+-----------------+---------------------+-------------+
|          2014|          Europe|          Ireland|                 Beer|         5.13|
|          2014|          Europe|          Ireland|            All types|        10.75|
|          2014|          Europe|          Ireland|              Spirits|         1.97|
|          2014|          Europe|          Ireland| Other alcoholic b...|         0.85|
|          2014|          Europe|          Ireland|                 Wine|         2.80|
+--------------+----------------+-----------------+---------------------+-------------+



In [0]:
df_whatever = df_whatever.withColumnRenamed("YEAR (DISPLAY)", "year").withColumnRenamed("REGION (DISPLAY)", "region").withColumnRenamed("COUNTRY (DISPLAY)", "country").withColumnRenamed("ALCOHOLTYPE (DISPLAY)", "alc_type").withColumnRenamed("Display Value", "value")

In [0]:
df_whatever.registerTempTable('df_whatever')

In [18]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)
sqlContext.sql('select * from df_whatever').show(50)

+----+--------------------+--------------------+--------------------+-----+
|year|              region|             country|            alc_type|value|
+----+--------------------+--------------------+--------------------+-----+
|2011|     Western Pacific|           Australia|                Wine| 3.79|
|2006|              Europe|          Kazakhstan|                Beer| 2.22|
|2010|              Africa|United Republic o...|           All types| 4.19|
|2012|     Western Pacific|         New Zealand|           All types| 9.20|
|2005|              Africa|             Algeria|             Spirits| 0.02|
|2003|              Europe|         Switzerland|                Beer| 3.37|
|2009|              Africa|Central African R...|                Wine| 0.01|
|2004|              Europe|             Ukraine|           All types| 6.79|
|2011|            Americas|             Ecuador|             Spirits| 1.15|
|2003|              Africa|              Guinea|             Spirits| 0.00|
|2013|      

In [19]:
type(spark)

pyspark.sql.session.SparkSession

In [20]:
sqlContext.sql('select min(year), max(year) from df_whatever').show()

+---------+---------+
|min(year)|max(year)|
+---------+---------+
|     2000|     2015|
+---------+---------+



In [21]:
sqlContext.sql('select min(year), max(year) from df_whatever where country=\'Ireland\'').show()

+---------+---------+
|min(year)|max(year)|
+---------+---------+
|     2000|     2014|
+---------+---------+



In [22]:
sqlContext.sql('select * from df_whatever where region=\'Europe\' and alc_type=\'All types\' order by value desc').show()

+----+------+----------+---------+-------+
|year|region|   country| alc_type|  value|
+----+------+----------+---------+-------+
|2009|Europe|San Marino|All types|No data|
|2009|Europe|    Monaco|All types|No data|
|2010|Europe|    Monaco|All types|No data|
|2000|Europe|    Monaco|All types|No data|
|2006|Europe|San Marino|All types|No data|
|2002|Europe|San Marino|All types|No data|
|2003|Europe|San Marino|All types|No data|
|2003|Europe|    Monaco|All types|No data|
|2000|Europe|San Marino|All types|No data|
|2006|Europe|    Monaco|All types|No data|
|2008|Europe|    Monaco|All types|No data|
|2007|Europe|    Monaco|All types|No data|
|2007|Europe|San Marino|All types|No data|
|2005|Europe|San Marino|All types|No data|
|2004|Europe|    Monaco|All types|No data|
|2004|Europe|San Marino|All types|No data|
|2001|Europe|San Marino|All types|No data|
|2005|Europe|    Monaco|All types|No data|
|2010|Europe|San Marino|All types|No data|
|2001|Europe|    Monaco|All types|No data|
+----+-----

In [0]:
df_whatever = sqlContext.sql("select * from df_whatever where value<>\'No data\' ")

In [24]:
sqlContext.sql('select * from df_whatever where region=\'Europe\' and year=2014 and alc_type=\'All types\' and value!=\'No data\' order by value desc').show()

+----+------+--------------------+---------+-----+
|year|region|             country| alc_type|value|
+----+------+--------------------+---------+-----+
|2014|Europe| Republic of Moldova|All types| 9.99|
|2014|Europe|            Portugal|All types| 9.88|
|2014|Europe|             Denmark|All types| 9.64|
|2014|Europe|         Switzerland|All types| 9.61|
|2014|Europe|              Serbia|All types| 9.09|
|2014|Europe|             Finland|All types| 8.80|
|2014|Europe|               Malta|All types| 8.49|
|2014|Europe|             Ukraine|All types| 8.06|
|2014|Europe|               Italy|All types| 7.56|
|2014|Europe|              Greece|All types| 7.53|
|2014|Europe|             Iceland|All types| 7.45|
|2014|Europe|              Sweden|All types| 7.30|
|2014|Europe|          Kazakhstan|All types| 6.29|
|2014|Europe|             Georgia|All types| 6.13|
|2014|Europe|              Norway|All types| 6.06|
|2014|Europe|             Albania|All types| 4.51|
|2014|Europe|Bosnia and Herzeg.

# RDD

Parallelized Collections



Parallelized collections are created by calling SparkContext’s parallelize method on an existing iterable or collection in your driver program. The elements of the collection are copied to form a distributed dataset that can be operated on in parallel. For example, here is how to create a parallelized collection holding the numbers 1 to 5:

In [0]:
data = [1, 2, 3, 4, 5]
distData = spark.parallelize(data)
distData.filter(lambda x: x % 2 == 0).collect()

In [0]:
textFile  = sc.textFile("file:///Users/Administrator/Downloads/war-and-peace.txt") 

In [0]:
textFile.count()

In [0]:
import functools
lineLengths = textFile.map(lambda s: len(s))
totalLength = lineLengths.reduce(lambda a, b: a + b)

In [0]:
print(totalLength)